## 문제 1. 
- 다음은 Used car 데이터 세트이다. 주어진 훈련 데이터 세트를 활용하여 중고차의 판매 가격을 예측하고 해당 예측 결과를 csv 파일로 제출하시오.
- 결과 제출 양식 : 제출한 예측값의 rmse 평가지표 값을 통해 영역별 배점에 따라 최종 점수가 반영될 예정

In [1]:
import pandas as pd 
pd.DataFrame({
    "ID"  : [1, 2, 3],
    "y_pred" : [12500, 16500, 11000]
})

,ID,y_pred
0,1,12500
1,2,16500
2,3,11000


In [2]:
import pandas as pd 
X_test = pd.read_csv("data/5회/504_x_test.csv")
X_train = pd.read_csv("data/5회/504_x_train.csv")
y_train = pd.read_csv("data/5회/504_y_train.csv")

In [3]:
import pandas as pd 
pd.DataFrame({
    "변수" : list(X_train.columns) + ["price"], 
    "설명" : [
        "중고차 ID 번호", 
        "차량 모델명", 
        "차량 등록 연도", 
        "변속기 종류", 
        "주행 거리", 
        "엔진 연료 종류",
        "도로세", 
        "갤런당 마일(miles per gallon), 연비", 
        "엔진 크기",   
        "가격(파운드)"
    ]
})

,변수,설명
0,id,중고차 ID 번호
1,model,차량 모델명
2,year,차량 등록 연도
3,transmission,변속기 종류
4,mileage,주행 거리
5,fuelType,엔진 연료 종류
6,tax,도로세
7,mpg,"갤런당 마일(miles per gallon), 연비"
8,engineSize,엔진 크기
9,price,가격(파운드)


## 데이터탐색

In [4]:
X_train.head()

,id,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,1,A1,2019,Automatic,3500,Petrol,145,40.9,2.0
1,2,RS4,2020,Semi-Auto,2500,Petrol,145,28.8,2.9
2,3,A8,2019,Semi-Auto,500,Diesel,145,40.4,3.0
3,4,Q5,2019,Semi-Auto,5089,Diesel,150,38.2,2.0
4,5,A5,2020,Semi-Auto,4951,Diesel,145,51.4,2.0


In [5]:
y_train.head()

,id,price
0,1,21350
1,2,69691
2,3,42950
3,4,31470
4,5,27495


In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            7468 non-null   int64  
 1   model         7468 non-null   object 
 2   year          7468 non-null   int64  
 3   transmission  7468 non-null   object 
 4   mileage       7468 non-null   int64  
 5   fuelType      7468 non-null   object 
 6   tax           7468 non-null   int64  
 7   mpg           7468 non-null   float64
 8   engineSize    7468 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 525.2+ KB


In [7]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7468 entries, 0 to 7467
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      7468 non-null   int64
 1   price   7468 non-null   int64
dtypes: int64(2)
memory usage: 116.8 KB


In [8]:
X_train.isnull().sum()

id              0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
dtype: int64

In [9]:
y_train.isnull().sum()

id       0
price    0
dtype: int64

In [10]:
len(X_train['id'].unique())

7468

In [11]:
len(y_train['id'].unique())

7468

## 데이터셋 분리

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5974, 9), (1494, 9), (5974, 2), (1494, 2))

## 컬럼 분리

In [13]:
X_train.pop('id')
y_train.pop('id')
X_test_id = X_test.pop('id')
y_test.pop('id')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5974, 8), (1494, 8), (5974, 1), (1494, 1))

## 데이터타입별로 분리

In [14]:
object_df = X_train.select_dtypes(include=object)
object_df.head()

,model,transmission,fuelType
2516,Q5,Semi-Auto,Diesel
434,Q3,Automatic,Diesel
751,A3,Manual,Petrol
5827,Q3,Manual,Petrol
1616,A7,Semi-Auto,Diesel


In [15]:
import numpy as np

number_df = X_train.select_dtypes(include=np.number)
number_df.head()

,year,mileage,tax,mpg,engineSize
2516,2016,39015,205,47.1,2.0
434,2019,6500,150,48.7,2.0
751,2019,5704,145,49.6,1.0
5827,2017,16100,150,49.6,1.4
1616,2016,21023,145,54.3,3.0


In [16]:
number_df.describe()

,year,mileage,tax,mpg,engineSize
count,5974.000000,5974.000000,5974.000000,5974.000000,5974.000000
mean,2017.092233,24855.027954,126.100603,50.908202,1.926599
std,2.125880,23204.113163,67.119139,12.655898,0.598562
min,1998.000000,1.000000,0.000000,19.300000,0.000000
25%,2016.000000,6000.000000,125.000000,40.900000,1.500000
50%,2017.000000,19218.500000,145.000000,50.000000,2.000000
75%,2019.000000,36627.500000,145.000000,58.900000,2.000000
max,2020.000000,168017.000000,580.000000,188.300000,6.300000


In [17]:
for column in object_df.columns:
    print(object_df[column].value_counts())
    print()

 A3     1085
 Q3      808
 A4      786
 A1      746
 Q5      491
 A5      474
 Q2      467
 A6      418
 Q7      207
 TT      193
 A7       69
 A8       66
 Q8       42
 RS6      24
 RS4      19
 RS5      19
 RS3      17
 R8       12
 SQ5      10
 S4        7
 SQ7       4
 S3        4
 S8        3
 S5        2
 RS7       1
Name: model, dtype: int64

Manual       2447
Semi-Auto    2042
Automatic    1485
Name: transmission, dtype: int64

Diesel    3177
Petrol    2783
Hybrid      14
Name: fuelType, dtype: int64



### 컬럼 삭제

In [18]:
object_df.pop('model')
X_train.pop('model')
X_test.pop('model')

6624      Q2
6483      A1
1372      S3
3298      A5
1041      A5
        ... 
410       A3
6244      A6
4043      A3
438       A1
3159     RS4
Name: model, Length: 1494, dtype: object

## 모델 생성

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder
from lightgbm import LGBMRegressor

transformer = ColumnTransformer([
    ('robust_scaler', RobustScaler(), ['mileage']),
    ('minmax_scaler', MinMaxScaler(), number_df.columns.drop('mileage')),
    ('ohc_encoder', OneHotEncoder(), object_df.columns)
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', transformer),
    ('model', LGBMRegressor(random_state=42, max_depth=5, learning_rate=0.05))
])

pipeline.fit(X_train, y_train)

D:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('robust_scaler',
                                                  RobustScaler(), ['mileage']),
                                                 ('minmax_scaler',
                                                  MinMaxScaler(),
                                                  Index(['year', 'tax', 'mpg', 'engineSize'], dtype='object')),
                                                 ('ohc_encoder',
                                                  OneHotEncoder(),
                                                  Index(['transmission', 'fuelType'], dtype='object'))])),
                ('model',
                 LGBMRegressor(learning_rate=0.05, max_depth=5,
                               random_state=42))])

## 모델 평가

In [20]:
from sklearn.metrics import mean_squared_error

def get_scores(model, X_train, X_test, y_train, y_test):
    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)
    train_score = np.sqrt(mean_squared_error(y_train, train_predict))
    test_score = np.sqrt(mean_squared_error(y_test, test_predict))
    return f'train : {train_score}, test : {test_score}'

get_scores(pipeline, X_train, X_test, y_train, y_test)

'train : 2863.987560701361, test : 3195.823033803084'

## 결과 제출

In [21]:
final_predict = pipeline.predict(X_test).round()
final_predict

array([26558., 10776., 21145., ..., 31661., 12040., 65855.])

In [22]:
result = pd.DataFrame({'ID':X_test_id, 'y_pred':final_predict})
result

,ID,y_pred
6624,6625,26558.0
6483,6484,10776.0
1372,1373,21145.0
3298,3299,31043.0
1041,1042,28582.0
...,...,...
410,411,21282.0
6244,6245,19151.0
4043,4044,31661.0
438,439,12040.0


In [23]:
result.to_csv('4.csv', index=False)